Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [x] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [x] Engineer at least two new features. (See below for explanation & ideas.)
- [x] Fit a linear regression model with at least two features.
- [x] Get the model's coefficients and intercept.
- [x] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [x] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [x] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [2]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [3]:
# take a look at the data
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,...,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,...,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,...,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,...,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,...,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.dtypes

bathrooms               float64
bedrooms                  int64
created                  object
description              object
display_address          object
latitude                float64
longitude               float64
price                     int64
street_address           object
interest_level           object
elevator                  int64
cats_allowed              int64
hardwood_floors           int64
dogs_allowed              int64
doorman                   int64
dishwasher                int64
no_fee                    int64
laundry_in_building       int64
fitness_center            int64
pre-war                   int64
laundry_in_unit           int64
roof_deck                 int64
outdoor_space             int64
dining_room               int64
high_speed_internet       int64
balcony                   int64
swimming_pool             int64
new_construction          int64
terrace                   int64
exclusive                 int64
loft                      int64
garden_p

In [5]:
# make a new column and conver the objects from created into a datetime object
df["dt"]=pd.to_datetime(df["created"],infer_datetime_format=True)

# make a total rooms ratio
df["room_ratio"]=(df['bedrooms']+df['bathrooms'])/2

In [6]:
df['room_ratio'].value_counts().sort_index()

0.00      151
0.50     9200
0.75        9
1.00    15286
1.25      154
1.50    11083
1.75      210
2.00     6940
2.25      291
2.50     3160
2.75      190
3.00     1605
3.25       67
3.50      277
3.75       32
4.00       83
4.25       11
4.50       49
5.00       16
5.50        1
6.00        2
Name: room_ratio, dtype: int64

In [7]:
#seperate just 2016 entries
df2016=df[df['dt'].dt.year==2016]

In [8]:
# take a look at what months we have to drop
df2016['dt'].dt.month.value_counts()

6    16973
4    16217
5    15627
Name: dt, dtype: int64

In [9]:
# well thats convenitent go ahead and train test split based on months from 2016
train=df2016[(df2016['dt'].dt.month==4) | (df2016['dt'].dt.month==5)]
# 'drop all that are in train and return what is left(june in this case)'
test=df2016.drop(train.index)

In [10]:
# make sure that my split is correct
print(train['dt'].dt.month.value_counts())
test['dt'].dt.month.value_counts()

4    16217
5    15627
Name: dt, dtype: int64


6    16973
Name: dt, dtype: int64

In [11]:
# import the Linear Regression model from sklearn
from sklearn.linear_model import LinearRegression
model=LinearRegression(n_jobs=-1)

In [12]:
# select features
# one of the reasons that i select the following features is because they would be my real world consiterations while looking 
# for an apartment for rent
features=['room_ratio','cats_allowed','laundry_in_unit','high_speed_internet']
# selecting the target to predict
target='price'


In [13]:
# fit the model with the selected features
model=model.fit(train[features],train[target])

In [14]:
# just for giggles make a prediction with the fitted model
model.predict([[1,1,1,1]])

array([3766.58227793])

In [15]:
from sklearn.metrics import mean_absolute_error,mean_squared_error
from math import sqrt
# make a funtion to calc RMSE
def RMSE(y_true,y_pred):
    '''
        this function caluclates Root Mean Squared Error (RMSE)
        @y_true: true observed values
        @y_pred: predicted values
    '''
    return sqrt(mean_squared_error(y_true=y_true,y_pred=y_pred))
def calc_errors(y_true,y_pred,df):
    '''
        Run a few error rate test's on a model's predictions and return them in a tuple or list
        runs: MAE,RMSE,R^2 tests
        @y_true: true observed values
        @y_pred: predicted values
    '''
    mae=mean_absolute_error(y_true=y_true,y_pred=y_pred)
    rmse=RMSE(y_true,y_pred)
    r2=model.score(df[features],df[target])
    print(f"Model R^2: {round(r2*100,2)}%\n"+
          f"Model Mean Absolute Error: {round(mae,2)}\n"+
          f"Model Root Mean Squared Error: {round(rmse,2)}\n",end='')
    
    return r2,mae,rmse

In [16]:
print("TRAIN ERRORS")
# make predictions on our test set
y_pred=model.predict(train[features])
y_true=train[target]
# calculate error rates
calc_errors(y_true,y_pred,train); # sqwelt return
print("\n")
print("TEST ERRORS")
# make predictions on our test set
y_pred=model.predict(test[features])
y_true=test[target]
# calculate error rates
calc_errors(y_true,y_pred,test); # sqwelt return
print()
print(f"The Coefficants are:{model.coef_}, and the Intercept is: {model.intercept_}")

TRAIN ERRORS
Model R^2: 44.87%
Model Mean Absolute Error: 875.73
Model Root Mean Squared Error: 1308.42


TEST ERRORS
Model R^2: 44.43%
Model Mean Absolute Error: 893.8
Model Root Mean Squared Error: 1314.21

The Coefficants are:[1541.51327942  178.94713577  737.05484732   59.38094286], and the Intercept is: 1249.6860725591105


I tried doing Principal Component Regression and couldn't get it to work, idk ill look into it further. 